In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os

os.environ['CLMM_MODELING_BACKEND'] = 'nc' 
import clmm
import pyccl as ccl

In [ ]:
print(clmm.__version__)
print(ccl.__version__)

In [ ]:
cosmo = clmm.Cosmology(H0=70.0, Omega_dm0=0.27-0.045, Omega_b0=0.045, Omega_k0=0.0)


In [ ]:
from clmm.theory.ccl import CCLCLMModeling
cosmoccl =  clmm.cosmology.ccl.CCLCosmology(H0=70.0, Omega_dm0=0.27-0.045, Omega_b0=0.045, Omega_k0=0.0)
moo = CCLCLMModeling(massdef='mean', delta_mdef=200, halo_profile_model='nfw', )

moo.set_cosmo(cosmoccl)
moo.set_concentration(4)
moo.set_mass(1.e15)

z_cl = 1.

#source properties
z_source = 2. #all sources in the same plane
z_distrib_func = clmm.utils._chang_z_distrib #sources redshift following a distribution
alpha = [2, -0.5]  

In [ ]:
r3d = np.logspace(-3,1, 1000)

rho = moo.eval_3d_density(r3d, z_cl)
Sigma = moo.eval_surface_density(r3d, z_cl)
DeltaSigma = moo.eval_excess_surface_density(r3d, z_cl)
Sigmac = moo.eval_critical_surface_density(z_cl, z_source)

# require CCL>=2.5.0 to use backend-specific calculations for the below methods
gammat = moo.eval_tangential_shear(r3d, z_cl, z_source)
kappa = moo.eval_convergence(r3d, z_cl, z_source)
gt = moo.eval_reduced_tangential_shear(r3d, z_cl, z_source)
mu = moo.eval_magnification(r3d, z_cl, z_source)
mu_bias = moo.eval_magnification_bias(r3d, z_cl, z_source, alpha)

In [ ]:
# force CLMM to use old methods if CCL>=2.5.0 and catch the results
ccl.__version__='2.4.0'

gammat_old = moo.eval_tangential_shear(r3d, z_cl, z_source)
kappa_old = moo._eval_convergence_core(r3d, z_cl, z_source)
gt_old = moo._eval_reduced_tangential_shear_sp_core(r3d, z_cl, z_source)
mu_old = moo._eval_magnification_core(r3d, z_cl, z_source)


In [ ]:
# compare the results from new and old methods
print('Max relative differences between calculations for new and old CCL')
print(f'gammat: {abs(gammat_old/gammat-1).max()}')
print(f'kappa: {abs(kappa_old/kappa-1).max()}')
print(f'gt: {abs(gt_old/gt-1).max()}')
print(f'mu: {abs(mu_old/mu-1).max()}')
fig = plt.figure(figsize=(8,8))
fig.gca().plot(r3d, abs(gammat_old/gammat-1))
fig.gca().plot(r3d, abs(kappa_old/kappa-1))
fig.gca().plot(r3d, abs(gt_old/gt-1))
fig.gca().plot(r3d, abs(mu_old/mu-1))
plt.show()

In [ ]:
moo2 = clmm.Modeling(massdef='mean', delta_mdef=200, halo_profile_model='nfw')

moo2.set_cosmo(cosmo)
moo2.set_concentration(4)
moo2.set_mass(1.e15)

z_cl = 1.

#source properties
z_source2 = 2 #all sources in the same plane
z_distrib_func2 = clmm.utils._chang_z_distrib #sources redshift following a distribution
alpha2 = [2, -0.5] 

In [ ]:
r3d_nc = np.logspace(-3, 1, 1000)
rho_nc = moo2.eval_3d_density(r3d_nc, z_cl)
Sigma_nc = moo2.eval_surface_density(r3d_nc, z_cl)
DeltaSigma_nc = moo2.eval_excess_surface_density(r3d_nc, z_cl)
Sigmac_nc = moo2.eval_critical_surface_density(z_cl, z_source2)


gammat_nc = moo2.eval_tangential_shear(r3d_nc, z_cl, z_source2)
kappa_nc = moo2.eval_convergence(r3d_nc, z_cl, z_source2)
gt_nc = moo2.eval_reduced_tangential_shear(r3d_nc, z_cl, z_source2)
mu_nc = moo2.eval_magnification(r3d_nc, z_cl, z_source2)
mu_bias_nc = moo2.eval_magnification_bias(r3d_nc, z_cl, z_source2, alpha2)

In [ ]:
#NC results
mu_dif = []
kappa_dif = []
gt_dif = []
mb_dif = []

plt.plot(r3d, abs(mu/mu_nc - 1), '-', label = 'Magnfication')
plt.plot(r3d, abs(kappa/kappa_nc -1), '-', label = 'Convergence')
plt.plot(r3d, abs(gammat/gammat_nc -1), '-', label = 'Tangential Shear')
plt.plot(r3d, abs(gt/gt_nc -1), '-', label = 'Reduced tangential shear')
plt.plot(r3d, abs(mu_bias[0]/mu_bias_nc[0] -1), '-', color='yellow', label = 'magnification bias')
plt.xscale('log')
plt.xlabel('Projected Radius')
plt.ylabel('Relative Error')
plt.yscale('log')
plt.legend()
plt.show()


In [ ]:
#Indexes gives me the values where the magnification is negative, which returns nan for the magnification bias
indexes = np.where(mu < 0)
a = indexes[0][0]
b = indexes[0][-1]
r_negative = r3d[a:b]
mu_bias_nan = mu_bias[0][a:b]
print(r_negative, mu_bias_nan)